In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn import ensemble
from sklearn import model_selection,metrics,tree,preprocessing
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,LSTM

## Paths

In [ ]:
TRAIN='/content/fulfilment_center_info.csv'
MEAL='/content/meal_info.csv'
CENTER='/content/train.csv'

## Reading the Files

In [ ]:
df_train=pd.read_csv(TRAIN)
df_train.head(10)

,center_id,city_code,region_code,center_type,op_area
0,11,679,56,TYPE_A,3.7
1,13,590,56,TYPE_B,6.7
2,124,590,56,TYPE_C,4.0
3,66,648,34,TYPE_A,4.1
4,94,632,34,TYPE_C,3.6
5,64,553,77,TYPE_A,4.4
6,129,593,77,TYPE_A,3.9
7,139,693,34,TYPE_C,2.8
8,88,526,34,TYPE_A,4.1
9,143,562,77,TYPE_B,3.8


In [ ]:
df_meal=pd.read_csv(MEAL)
df_meal.head(10)

,meal_id,category,cuisine
0,1885,Beverages,Thai
1,1993,Beverages,Thai
2,2539,Beverages,Thai
3,1248,Beverages,Indian
4,2631,Beverages,Indian
5,1311,Extras,Thai
6,1062,Beverages,Italian
7,1778,Beverages,Italian
8,1803,Extras,Thai
9,1198,Extras,Thai


In [ ]:
df_center=pd.read_csv(CENTER)
df_center.head(10)

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders
0,1379560,1,55,1885,136.83,152.29,0,0,177
1,1466964,1,55,1993,136.83,135.83,0,0,270
2,1346989,1,55,2539,134.86,135.86,0,0,189
3,1338232,1,55,2139,339.50,437.53,0,0,54
4,1448490,1,55,2631,243.50,242.50,0,0,40
5,1270037,1,55,1248,251.23,252.23,0,0,28
6,1191377,1,55,1778,183.36,184.36,0,0,190
7,1499955,1,55,1062,182.36,183.36,0,0,391
8,1025244,1,55,2707,193.06,192.06,0,0,472
9,1054194,1,55,1207,325.92,384.18,0,1,676



### So Here We see that the main Aim is to calculate the no of orders
#### We can see that df_center could map the feature of meals from df_meal using the meal as the common key
#### Similarily We could all map the features of center to the the training set using center as the foreign key

## Drop Unnecessary Feature from center and map the Features from df_meal

In [ ]:
df_center=df_center.merge(df_meal,on='meal_id',how='inner')
df_center.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,category,cuisine
0,1379560,1,55,1885,136.83,152.29,0,0,177,Beverages,Thai
1,1040403,1,24,1885,136.83,136.83,0,0,1498,Beverages,Thai
2,1103215,1,11,1885,136.83,136.83,0,0,959,Beverages,Thai
3,1034383,1,83,1885,121.31,120.31,0,1,1025,Beverages,Thai
4,1118999,1,32,1885,114.52,113.52,0,1,445,Beverages,Thai


In [ ]:
df_meall=df_meal.set_index('meal_id').T.to_dict()
print(df_meall)

{1885: {'category': 'Beverages', 'cuisine': 'Thai'}, 1993: {'category': 'Beverages', 'cuisine': 'Thai'}, 2539: {'category': 'Beverages', 'cuisine': 'Thai'}, 1248: {'category': 'Beverages', 'cuisine': 'Indian'}, 2631: {'category': 'Beverages', 'cuisine': 'Indian'}, 1311: {'category': 'Extras', 'cuisine': 'Thai'}, 1062: {'category': 'Beverages', 'cuisine': 'Italian'}, 1778: {'category': 'Beverages', 'cuisine': 'Italian'}, 1803: {'category': 'Extras', 'cuisine': 'Thai'}, 1198: {'category': 'Extras', 'cuisine': 'Thai'}, 2707: {'category': 'Beverages', 'cuisine': 'Italian'}, 1847: {'category': 'Soup', 'cuisine': 'Thai'}, 1438: {'category': 'Soup', 'cuisine': 'Thai'}, 2494: {'category': 'Soup', 'cuisine': 'Thai'}, 2760: {'category': 'Other Snacks', 'cuisine': 'Thai'}, 2490: {'category': 'Salad', 'cuisine': 'Italian'}, 1109: {'category': 'Rice Bowl', 'cuisine': 'Indian'}, 2290: {'category': 'Rice Bowl', 'cuisine': 'Indian'}, 1525: {'category': 'Other Snacks', 'cuisine': 'Thai'}, 2704: {'categ

In [ ]:
df_center.drop('meal_id',axis=1,inplace=True)

In [ ]:
df_train=df_train.merge(df_center,on='center_id',how='inner')
df_train.head()

,center_id,city_code,region_code,center_type,op_area,id,week,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,category,cuisine
0,11,679,56,TYPE_A,3.7,1103215,1,136.83,136.83,0,0,959,Beverages,Thai
1,11,679,56,TYPE_A,3.7,1091356,2,133.86,135.86,0,0,1054,Beverages,Thai
2,11,679,56,TYPE_A,3.7,1195933,3,135.86,133.86,0,0,891,Beverages,Thai
3,11,679,56,TYPE_A,3.7,1425602,4,134.86,135.86,0,0,1107,Beverages,Thai
4,11,679,56,TYPE_A,3.7,1249127,5,146.53,146.53,0,0,919,Beverages,Thai


In [ ]:
df_train.drop(['center_id','city_code','region_code','id'],axis=1,inplace=True)
df_train.shape

(456548, 10)

In [ ]:
TEST='/content/test_QoiMO9B.csv'
df_test=pd.read_csv(TEST)
df_test.head(10)

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured
0,1028232,146,55,1885,158.11,159.11,0,0
1,1127204,146,55,1993,160.11,159.11,0,0
2,1212707,146,55,2539,157.14,159.14,0,0
3,1082698,146,55,2631,162.02,162.02,0,0
4,1400926,146,55,1248,163.93,163.93,0,0
5,1284113,146,55,1778,190.15,190.15,0,0
6,1197966,146,55,1062,191.09,192.09,0,0
7,1132739,146,55,2707,242.56,240.56,0,0
8,1057981,146,55,1207,360.90,360.90,0,0
9,1095932,146,55,1230,383.18,384.18,0,0


In [ ]:
category=[]
cuisine=[]
meal=df_test['meal_id']
for i in meal:
  arr=df_meall[i]
  category.append(arr['category'])
  cuisine.append(arr['cuisine'])
df_test['category']=category
df_test['cuisine']=cuisine
df_test.drop('meal_id',axis=1,inplace=True)
df_test.head(10)

,id,week,center_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,category,cuisine
0,1028232,146,55,158.11,159.11,0,0,Beverages,Thai
1,1127204,146,55,160.11,159.11,0,0,Beverages,Thai
2,1212707,146,55,157.14,159.14,0,0,Beverages,Thai
3,1082698,146,55,162.02,162.02,0,0,Beverages,Indian
4,1400926,146,55,163.93,163.93,0,0,Beverages,Indian
5,1284113,146,55,190.15,190.15,0,0,Beverages,Italian
6,1197966,146,55,191.09,192.09,0,0,Beverages,Italian
7,1132739,146,55,242.56,240.56,0,0,Beverages,Italian
8,1057981,146,55,360.90,360.90,0,0,Beverages,Continental
9,1095932,146,55,383.18,384.18,0,0,Beverages,Continental


In [ ]:
df_center=pd.read_csv(TRAIN)
df_center=df_center.set_index('center_id').T.to_dict()
len(df_center)

77

In [ ]:
center_type=[]
op_area=[]
center=df_test['center_id']
for i in center:
  arr=df_center[i]
  center_type.append(arr['center_type'])
  op_area.append(arr['op_area'])
df_test['center_type']=center_type
df_test['op_area']=op_area
df_test.drop('center_id',axis=1,inplace=True)
df_test.head(10)

,id,week,checkout_price,base_price,emailer_for_promotion,homepage_featured,category,cuisine,center_type,op_area
0,1028232,146,158.11,159.11,0,0,Beverages,Thai,TYPE_C,2.0
1,1127204,146,160.11,159.11,0,0,Beverages,Thai,TYPE_C,2.0
2,1212707,146,157.14,159.14,0,0,Beverages,Thai,TYPE_C,2.0
3,1082698,146,162.02,162.02,0,0,Beverages,Indian,TYPE_C,2.0
4,1400926,146,163.93,163.93,0,0,Beverages,Indian,TYPE_C,2.0
5,1284113,146,190.15,190.15,0,0,Beverages,Italian,TYPE_C,2.0
6,1197966,146,191.09,192.09,0,0,Beverages,Italian,TYPE_C,2.0
7,1132739,146,242.56,240.56,0,0,Beverages,Italian,TYPE_C,2.0
8,1057981,146,360.90,360.90,0,0,Beverages,Continental,TYPE_C,2.0
9,1095932,146,383.18,384.18,0,0,Beverages,Continental,TYPE_C,2.0


In [ ]:
id=df_test['id']
df_test.drop('id',axis=1,inplace=True)
df_test.shape

(32573, 9)

In [ ]:
df_train.head(3)

,center_type,op_area,week,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,category,cuisine
0,TYPE_A,3.7,1,136.83,136.83,0,0,959,Beverages,Thai
1,TYPE_A,3.7,2,133.86,135.86,0,0,1054,Beverages,Thai
2,TYPE_A,3.7,3,135.86,133.86,0,0,891,Beverages,Thai


In [ ]:
df_test.head(3)

,week,checkout_price,base_price,emailer_for_promotion,homepage_featured,category,cuisine,center_type,op_area
0,146,158.11,159.11,0,0,Beverages,Thai,TYPE_C,2.0
1,146,160.11,159.11,0,0,Beverages,Thai,TYPE_C,2.0
2,146,157.14,159.14,0,0,Beverages,Thai,TYPE_C,2.0


In [188]:
df_train.to_csv('training_set.csv')
df_test.to_csv('testing_set.csv')
id.to_csv('test_ids.csv')